This contains common code blocks used for configuration and user input
New tools are developed here.

In [ ]:
# Install our library 
!pip install CanvasHacks


In [ ]:
%cd ~/Dropbox/CanvasHacks

# %cd ../
from IPython.display import display
from ipywidgets import widgets

from CanvasHacks import environment
from CanvasHacks.RequestTools import get_all_course_assignments, get_assignments_needing_grading
from CanvasHacks.Configuration import InteractiveConfiguration
from CanvasHacks.InputFields import make_course_ids_input, make_canvas_token_input, make_canvas_url_input, make_general_reset_button
import CanvasHacks.GradingTools as GT
import CanvasHacks.DownloadProcessingTools as PT
from CanvasHacks.DataManagement import DataStore

from CanvasHacks.ConsolidatedTextOutput import make_assignment_header, make_consolidated_text_fields
from CanvasHacks.UploadGradeTools import make_upload_button

# This aren't used in the non-saving version
from CanvasHacks.FileTools import makeDataFileIterator
from CanvasHacks.JournalsFileTools import get_journal_folders, make_folder_list, calculate_journal_counts
from CanvasHacks.JournalsFileTools import journal_folder_name, create_folder


# Setup the configuration

In [ ]:
from CanvasHacks.CredentialFields import make_credentials_input

In [ ]:
make_credentials_input()

In [ ]:

def token_instructions_and_input():
    token_instructions = """
    <div>
        <div>
            <p>You'll need to generate a token for the Canvas API. The token replaces your username/password when making requests to Canvas.</p> 
            <p>Please record the token in a secure place. This program will not save it; you will have to re-enter it every time you use this tool.</p>

            <p>First log into Canvas. Then: </p>
            <blockquote>
                <p>Click the "profile" link in the top right menu bar, or navigate to /profile</p>

                <p>Under the "Approved Integrations" section, click the button to generate a new access token.</p>

            <p>Once the token is generated, you cannot view it again, and you'll have to generate a new token if you forget
                it.
                Remember that access tokens are password equivalent, so keep it secret (ed: keep it safe!).</p>

                <p><a href="https://canvas.instructure.com/doc/api/file.oauth.html#manual-token-generation">https://canvas.instructure.com/doc/api/file.oauth.html#manual-token-generation</a>
                </p>
            </blockquote>
        </div>
        <div>
            <h5> Paste your token into the box below. You don't have to click anything, its value will be read when
                needed</h5>
        </div>
    </div>
    """

    instruct = widgets.HTML(value=token_instructions)
    display(instruct)
    make_canvas_token_input()

def section_instructions_and_input():
    section_instruct = """
    <div>
        <p>You will also need the id number(s) of the courses you wish to grade. You can find them by logging into canvas
            and going to the course page. The number you are looking for will be in the url, immediately after
            '/courses/'. For example:</p>
        <blockquote>https://canvas.csun.edu/courses/<b>12345</b></blockquote>
        <p>Write this number down somewhere secure so that you can re-enter it next time you use the
            program.</p>

        <h5>Add the course ids one-at-a-time using the box and buttons below</h5>
    </div>
    """

    display(widgets.HTML(section_instruct))
    make_course_ids_input()



def make_credentials_input():
    token_instructions_and_input()
    section_instructions_and_input()

# Choose assignments

In [ ]:
from IPython.display import display
from ipywidgets import widgets

def assignment_buttons():
    # Get list of all assignments for the courses
    for course_id in environment.CONFIG.course_ids:
        assignments = get_all_course_assignments(course_id)
        assignments = [(a['id'], a['name']) for a in assignments]
        display(widgets.HTML(value="<h4>Course {}</h4>".format(course_id)))

        # Display a button for each
        for assignment_id, name in assignments:
            b = widgets.Button( description=name )
            def callback( change ):
                environment.CONFIG.add_assignment(assignment_id, name)
            b.on_click( callback )
            display( b )

In [ ]:
assignment_buttons()

## Reseting the configuration (when you're done!)

Closing your session will automatically wipe out the token and ids you've entered here. 

However, if you'd like to be sure that they've been removed, you can click the button below. The two cell's below the button should show you an empty list of ids (it looks like: [ ]) and that the token has been set to False

In [ ]:
make_general_reset_button()

In [ ]:
InteractiveConfiguration.course_ids

In [ ]:
InteractiveConfiguration.canvas_token

# Downloading files

In [ ]:
!pip install pandas

In [ ]:
import base64
from IPython.display import HTML
import pandas as pd

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    """
    https://medium.com/ibm-data-science-experience/how-to-upload-download-files-to-from-notebook-in-my-local-machine-6a4e65a15767
    """
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


In [ ]:
 df = pd.DataFrame({'AAA': [4, 5, 6, 7],
                 'BBB': [10, 20, 30, 40],
                    'CCC': [100, 50, -30, -50]})
    

create_download_link(df)

In [ ]:
def create_text_download_link( txt, title = "Download file", filename = "test.txt"):
    """Displays a link with which to download the text as a file"""
    payload = txt
    html = '<a download="{filename}" href="data:text/text,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


In [ ]:
test = "tacos are nom"
create_text_download_link(test)